In [1]:
import yfinance as yf
import numpy as np
import pandas as pd

In [2]:
port1_tickers = ["GLEN.L", "MRW.L", "AZN"]
port2_tickers = ["LGEN.L", "TSCO", "GSK"]

In [3]:
# the function aims to calculate a stock's daily return, 
# by giving the stock and its weight with in a portfolio
def stock_return(ticker, weight):
    stock_df = yf.download(ticker, start='2021-01-01', end='2021-10-15') # get data from yfinance
    # calculate the daily weighted return by using the daily close price
    daily_returns = weight * stock_df['Close'].pct_change()
    return daily_returns

In [4]:
# the function aims to transform the data into usable dataset that can be used for my following code
def transform_data(ticker, weight):
    stock_return_dict = dict()
    key_list = []
    val_list = []
    return_d = stock_return(ticker, weight).to_dict()
    for key in return_d.keys():
        key_list.append(key)
    stock_return_dict["Date"] = key_list
    for val in return_d.values():
        val_list.append(val)
    stock_return_dict[ticker + "'s" + " weighted return"] = val_list
    df = pd.DataFrame(stock_return_dict)
    return df # a two column dataframe with date in the first column, 
              # and the second column shows the stock's weighted return

In [5]:
# the function calculates the portfolio return
# input a list that comprises the name of the portfolio, and a list of the weight of a stock
def portfolio_return(tickers, weight_vector):
    df_list = []
    for ticker_index in range(len(tickers)):
        dframe = transform_data(tickers[ticker_index], weight_vector[ticker_index])
        df_list.append(dframe)
    df0 = df_list[0]
    df0["portfolio return"] = 0
    for i in range(1, len(df_list)):
        df0 = df0.merge(df_list[i], how='left', left_on = 'Date', right_on = 'Date')
        df0["portfolio return"] = df0["portfolio return"] + df0.iloc[:,i]
    pr = df0[['Date', 'portfolio return']]
    return pr # a two column dataframe with the first column records the date,
              # and the second column reveals the portfolio return

In [6]:
# the function calculates the standard deviation of negative asset returns
# input the list that contains the names of the portfolios,
# and the list that includes the weight of each stock
def negative_return_std(tickers, weight_vector):
    port_return = portfolio_return(tickers, weight_vector)
    negative_return = port_return[port_return['portfolio return'] < 0]
    standard_deviation = negative_return['portfolio return'].std()
    return standard_deviation 

In [7]:
# the function calculates the sortino ratio and return a dataframe with the corresponding date
# input the the list with the names within the portfolio,
# the list of the weight of each stock,
# and the sonia rate to calculate the sortino ratio
def sortino_ratio(tickers, weight_vector, sonia_rate):
    pf_return = portfolio_return(tickers, weight_vector)
    std = negative_return_std(tickers, weight_vector)
    sortino_ratio = (pf_return['portfolio return'] - sonia_rate)/std
    pf_return['sortino ratio'] = 0
    pf_return['sortino ratio'] =  sortino_ratio
    return pf_return['sortino ratio'] # a two column dataframe with the first column shows the date,
                                      # and the second column gives the sortino ratio on a given day

In [8]:
sortino_ratio(port1_tickers, [1/3, 1/3, 1/3], 0.000502)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


0           NaN
1      2.683771
2      5.983747
3      1.240458
4     -0.641329
         ...   
190   -0.727228
191    2.488678
192    0.784741
193    0.202352
194    2.549309
Name: sortino ratio, Length: 195, dtype: float64

In [9]:
sortino_ratio(port2_tickers, [1/3, 1/3, 1/3], 0.000502)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


0           NaN
1     -0.906119
2      5.391250
3      0.330255
4      0.593193
         ...   
194   -0.004185
195    0.285015
196   -0.804843
197   -0.100788
198   -0.165368
Name: sortino ratio, Length: 199, dtype: float64